In [1]:
from GPT_model import * 
import gc
import torch 
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from transformers import GPT2Config, GPT2Model
from tqdm.notebook import tqdm
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR

In [2]:
years_KOSPI = ['2013','2014','2015','2016','2017','2018','2019','2020','2021']
years_KOSDAQ = ['2013','2014','2015','2016','2017','2018','2019','2020','2021']



# years_KOSPI = ['2013','2014','2015','2016','2017','2018',]
# years_KOSDAQ = ['2013','2014','2015','2016','2017']


In [3]:
train_x_list = []
train_y_list = []
close2close_list = []

threshold = 0.3

# Loading KOSPI Data
print('---- LOAD KOSPI DATA ----')
for year in tqdm(years_KOSPI):
    train_x_temp = np.load(f'./data/KOSPI/new_normalize_data3/KOSPI_{year}_train_x_norm_close2close.npy')
    train_y_temp = np.load(f'./data/KOSPI/new_normalize_data3/KOSPI_{year}_train_y_norm_close2close.npy')
    
    train_x_temp = train_x_temp.astype(np.float32)
    train_y_temp = train_y_temp.astype(np.float32)
    
    mask = (train_y_temp != -float('inf')) & (train_y_temp != float('inf'))
    train_x_temp = train_x_temp[mask]
    train_y_temp = train_y_temp[mask]
    
    
    close2close = train_y_temp
    train_y_temp = (train_y_temp > threshold).astype(float)
    
    train_x_temp, train_y_temp, close2close = undersample_data(train_x_temp, train_y_temp,close2close)
    
    train_x_list.append(train_x_temp)
    train_y_list.append(train_y_temp)
    close2close_list.append(close2close)

    del train_x_temp, train_y_temp  # Explicitly delete temporary variables
    gc.collect()

# Loading KOSDAQ Data
print('---- LOAD KOSDAQ DATA ----')
for year in tqdm(years_KOSDAQ):
    train_x_temp = np.load(f'./data/KOSDAQ/new_normalize_data3/KOSDAQ_{year}_train_x_norm_close2close.npy')
    train_y_temp = np.load(f'./data/KOSDAQ/new_normalize_data3/KOSDAQ_{year}_train_y_norm_close2close.npy')
    
    train_x_temp = train_x_temp.astype(np.float32)
    train_y_temp = train_y_temp.astype(np.float32)
    
    mask = (train_y_temp != -float('inf')) & (train_y_temp != float('inf'))
    train_x_temp = train_x_temp[mask]
    train_y_temp = train_y_temp[mask]

    close2close = train_y_temp
    train_y_temp = (train_y_temp > threshold).astype(float)
    
    train_x_temp, train_y_temp, close2close = undersample_data(train_x_temp, train_y_temp,close2close)
    
    train_x_list.append(train_x_temp)
    train_y_list.append(train_y_temp)
    close2close_list.append(close2close)

    del train_x_temp, train_y_temp  # Explicitly delete temporary variables
    gc.collect()

# Convert list of arrays to single array
train_x_undersampled = np.concatenate(train_x_list, axis=0)
train_y_undersampled = np.concatenate(train_y_list, axis=0)
cls2cls_undersampled = np.concatenate(close2close_list, axis=0)

del train_x_list, train_y_list,close2close_list  # Delete the lists to free up memory
gc.collect()


print(f'Data all : {len(train_y_undersampled)}')
print(f'positive : {train_y_undersampled.sum()}')
print(f'negative : {len(train_y_undersampled)-train_y_undersampled.sum()}')


seed = 42

X_train, X_val, y_train, y_val, real_cl2cl_rate_train, real_cl2cl_rate_val = train_test_split(
    train_x_undersampled, 
    train_y_undersampled, 
    cls2cls_undersampled,
    test_size=0.2, 
    stratify=train_y_undersampled,
    random_state=seed
)

print('---- arange the data ----')

print(f'Data all : {len(y_train)}')
print(f'positive : {y_train.sum()}')
print(f'negative : {len(y_train)-y_train.sum()}')

---- LOAD KOSPI DATA ----


  0%|          | 0/9 [00:00<?, ?it/s]

---- LOAD KOSDAQ DATA ----


  0%|          | 0/9 [00:00<?, ?it/s]

Data all : 458482
positive : 229241.0
negative : 229241.0
---- arange the data ----
Data all : 366785
positive : 183393.0
negative : 183392.0


In [4]:
# Convert the arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
real_cl2cl_rate_train_tensor = torch.tensor(real_cl2cl_rate_train, dtype=torch.float32)  # Convert real_cl2cl_rate_train to a tensor

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
real_cl2cl_rate_val_tensor = torch.tensor(real_cl2cl_rate_val, dtype=torch.float32)  # Convert real_cl2cl_rate_val to a tensor

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor, real_cl2cl_rate_train_tensor)  # Include real_cl2cl_rate_train_tensor
val_dataset = TensorDataset(X_val_tensor, y_val_tensor, real_cl2cl_rate_val_tensor)  # Include real_cl2cl_rate_val_tensor

# Create DataLoaders
BATCH_SIZE  = 256
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
BALANCE = 100000000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
real_cl2cl_rate_train_tensor = torch.tensor(real_cl2cl_rate_train, dtype=torch.float32).to(device)
# Define hyperparameters
learning_rate = 1e-3
num_epochs = 30

# Initialize model and optimizer
embed_size = 42  # Number of features in your data
num_layers = 6  # Number of transformer blocks (this is a hyperparameter, you can adjust it)
heads = 6  # Number of attention heads (another hyperparameter)
forward_expansion = 1  # Expansion factor for the feed-forward layer
max_length = 60  # Your sequence length
dropout = 0.2  # Dropout rate

model = GPT(
    embed_size=embed_size,
    num_layers=num_layers,
    heads=heads,
    forward_expansion=forward_expansion,
    max_length=max_length,
    dropout=dropout

)
initialize_weights(model)
model.load_state_dict(torch.load('./model/layer6_new/Transformer_layer6_3.pt'))
model.to(device)
model.pe = model.pe.to(device)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=1e-5,
    amsgrad=True
)

# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# loss_fn = nn.MSELoss()  # Mean Squared Error Loss for regression tasks
# mae_loss = torch.nn.L1Loss()

scheduler = StepLR(optimizer, step_size=10, gamma=0.8)
criterion = nn.BCEWithLogitsLoss()  # Use this instead of mean squared error.

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

print_interval = len(train_loader) // 10  # Compute the interval for printing

for epoch in tqdm(range(num_epochs), total=num_epochs):
    
    model.train()
    running_loss = 0.0
    correct_train_predictions = 0  # Track correct predictions for training
    # train_average_profit = 0
    # total_invested = 0  # Track total invested balance
    # total_profit = 0  # Track total profit

    for batch_idx, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
        total_invested = 0  # Track total invested balance
        total_profit = 0  # Track total profit
        batch_start = batch_idx * BATCH_SIZE
        batch_end = batch_start + BATCH_SIZE
        inputs, targets,real_cl2cl_rates  = batch
        inputs, targets,real_cl2cl_rates = inputs.to(device), targets.to(device) , real_cl2cl_rates.to(device)
        
        # smoothed_targets = smooth_binary_labels_torch(targets, epsilon=0.1)
        # smoothed_targets = smoothed_targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        probabilities = torch.sigmoid(outputs)
        predictions = (probabilities > 0.5).float()

        # Compute training accuracy
        correct_train_predictions += (predictions.squeeze() == targets).sum().item()
        
        # Predicted investments
        threshold = 0.8
        invest = probabilities > threshold  # Boolean tensor
        invest_amount = torch.where(invest, BALANCE * probabilities, torch.tensor([0.0]).to(device))  # Invest based on confidence
        profit = invest_amount * real_cl2cl_rates  # Element-wise multiplication
        total_profit += profit.sum().item()  # Sum profit for the batch
        total_invested += invest_amount.sum().item() 

        epsilon = 1e-10
        # probabilities = probabilities.clamp(min=epsilon)

        loss = criterion(probabilities.view(-1), targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        # Check if it's time to print
        if (batch_idx + 1) % print_interval == 0:
            avg_loss = running_loss / print_interval
            train_accuracy = correct_train_predictions / (print_interval * train_loader.batch_size)
            
            # train_profit = train_average_profit / (print_interval * train_loader.batch_size)
            if total_invested == 0:
                train_average_profit = 0
            else:
                train_average_profit = ((total_profit - total_invested) / total_invested ) 

            
            # Validation loss and accuracy computation
            model.eval()
            val_loss = 0.0
            correct_val_predictions = 0  # Track correct predictions for validation
            val_average_profit = 0
            total_invested = 0  # Track total invested balance
            total_profit = 0  # Track total profit
            with torch.no_grad():
                for val_batch in val_loader:
                    inputs, targets,real_cl2cl_rates  = val_batch
                    inputs, targets,real_cl2cl_rates = inputs.to(device), targets.to(device),real_cl2cl_rates.to(device)
                    outputs = model(inputs)
                    probabilities = torch.sigmoid(outputs)
                    val_predictions = (probabilities > 0.5).float()
                    
                    invest = probabilities > threshold  # Boolean tensor
                    invest_amount = torch.where(invest, BALANCE * probabilities, torch.tensor([0.0]).to(device))  # Invest based on confidence
                    profit = invest_amount * real_cl2cl_rates  # Element-wise multiplication
                    total_profit += profit.sum().item()  # Sum profit for the batch
                    total_invested += invest_amount.sum().item() 
                    
                    # Compute validation accuracy
                    correct_val_predictions += (val_predictions.squeeze() == targets).sum().item()
                    val_loss += criterion(probabilities.squeeze(1), targets).item()
                    
            avg_val_loss = val_loss / len(val_loader)
            val_accuracy = correct_val_predictions / (len(val_loader) * val_loader.batch_size)
            # val_profit = val_average_profit / (len(val_loader) * val_loader.batch_size)
            if total_invested == 0:
                val_average_profit = 0
            else:
                val_average_profit = (total_profit - total_invested) / total_invested
            model.train()
            
            current_lr = optimizer.param_groups[0]['lr']

            print(f"Epoch: {epoch + 1}, Batch: {batch_idx + 1}, lr : {current_lr}")
            print(f"Train Loss: {avg_loss:.4f}, Train Acc: {train_accuracy*100:.2f}%, Train profit : {train_average_profit*100:.2f}%")
            print(f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy*100:.2f}%, Val profit : {val_average_profit*100:.2f}%")

            running_loss = 0.0
            correct_train_predictions = 0  # Reset the correct count for training
            train_average_profit = 0
    scheduler.step()


cuda


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/1433 [00:00<?, ?it/s]

Epoch: 1, Batch: 143, lr : 0.001
Train Loss: 0.6724, Train Acc: 58.86%, Train profit : 7081.99%
Val Loss: 0.6746, Val Acc: 61.87%, Val profit : 7612.64%
Epoch: 1, Batch: 286, lr : 0.001
Train Loss: 0.6711, Train Acc: 58.63%, Train profit : 8015.47%
Val Loss: 0.6697, Val Acc: 61.36%, Val profit : 7611.78%
Epoch: 1, Batch: 429, lr : 0.001
Train Loss: 0.6708, Train Acc: 59.62%, Train profit : 7336.46%
Val Loss: 0.6683, Val Acc: 60.02%, Val profit : 7612.46%
Epoch: 1, Batch: 572, lr : 0.001
Train Loss: 0.6709, Train Acc: 59.12%, Train profit : 8129.77%
Val Loss: 0.6685, Val Acc: 59.76%, Val profit : 7612.12%
Epoch: 1, Batch: 715, lr : 0.001
Train Loss: 0.6729, Train Acc: 58.87%, Train profit : 7374.32%
Val Loss: 0.6720, Val Acc: 61.27%, Val profit : 7610.59%
Epoch: 1, Batch: 858, lr : 0.001
Train Loss: 0.6697, Train Acc: 58.96%, Train profit : 7161.77%
Val Loss: 0.6694, Val Acc: 61.25%, Val profit : 7610.66%
Epoch: 1, Batch: 1001, lr : 0.001
Train Loss: 0.6698, Train Acc: 58.83%, Train pro

  0%|          | 0/1433 [00:00<?, ?it/s]

...

Epoch: 28, Batch: 143, lr : 0.00064
Train Loss: 0.6667, Train Acc: 60.99%, Train profit : 7402.82%
Val Loss: 0.6670, Val Acc: 63.16%, Val profit : 7608.30%
Epoch: 28, Batch: 286, lr : 0.00064
Train Loss: 0.6659, Train Acc: 60.68%, Train profit : 8197.89%
Val Loss: 0.6664, Val Acc: 62.91%, Val profit : 7608.12%
Epoch: 28, Batch: 429, lr : 0.00064
Train Loss: 0.6643, Train Acc: 61.04%, Train profit : 7808.13%
Val Loss: 0.6667, Val Acc: 63.28%, Val profit : 7609.02%
Epoch: 28, Batch: 572, lr : 0.00064
Train Loss: 0.6679, Train Acc: 60.04%, Train profit : 8424.90%
Val Loss: 0.6654, Val Acc: 62.30%, Val profit : 7608.10%
Epoch: 28, Batch: 715, lr : 0.00064
Train Loss: 0.6652, Train Acc: 60.96%, Train profit : 7119.38%
Val Loss: 0.6667, Val Acc: 63.20%, Val profit : 7608.52%
Epoch: 28, Batch: 858, lr : 0.00064
Train Loss: 0.6652, Train Acc: 61.06%, Train profit : 7178.89%
Val Loss: 0.6663, Val Acc: 63.04%, Val profit : 7608.30%
Epoch: 28, Batch: 1001, lr : 0.00064
Train Loss: 0.6676, Train A

  0%|          | 0/1433 [00:00<?, ?it/s]

Epoch: 29, Batch: 143, lr : 0.00064
Train Loss: 0.6659, Train Acc: 60.45%, Train profit : 7364.18%
Val Loss: 0.6660, Val Acc: 63.12%, Val profit : 7610.09%
Epoch: 29, Batch: 286, lr : 0.00064
Train Loss: 0.6664, Train Acc: 60.96%, Train profit : 7719.52%
Val Loss: 0.6670, Val Acc: 62.91%, Val profit : 7609.62%
Epoch: 29, Batch: 429, lr : 0.00064
Train Loss: 0.6675, Train Acc: 60.03%, Train profit : 7500.88%
Val Loss: 0.6662, Val Acc: 63.27%, Val profit : 7609.78%
Epoch: 29, Batch: 572, lr : 0.00064
Train Loss: 0.6644, Train Acc: 61.08%, Train profit : 7728.24%
Val Loss: 0.6654, Val Acc: 62.08%, Val profit : 7606.74%
Epoch: 29, Batch: 715, lr : 0.00064
Train Loss: 0.6650, Train Acc: 61.16%, Train profit : 7293.82%
Val Loss: 0.6682, Val Acc: 62.50%, Val profit : 7608.81%
Epoch: 29, Batch: 858, lr : 0.00064
Train Loss: 0.6673, Train Acc: 60.56%, Train profit : 7178.87%
Val Loss: 0.6653, Val Acc: 62.70%, Val profit : 7608.04%
Epoch: 29, Batch: 1001, lr : 0.00064
Train Loss: 0.6643, Train A

  0%|          | 0/1433 [00:00<?, ?it/s]

Epoch: 30, Batch: 143, lr : 0.00064
Train Loss: 0.6657, Train Acc: 60.75%, Train profit : 7801.71%
Val Loss: 0.6701, Val Acc: 63.48%, Val profit : 7608.63%
Epoch: 30, Batch: 286, lr : 0.00064
Train Loss: 0.6645, Train Acc: 60.85%, Train profit : 6942.90%
Val Loss: 0.6678, Val Acc: 62.77%, Val profit : 7611.59%
Epoch: 30, Batch: 429, lr : 0.00064
Train Loss: 0.6670, Train Acc: 60.49%, Train profit : 8297.05%
Val Loss: 0.6687, Val Acc: 63.40%, Val profit : 7608.33%
Epoch: 30, Batch: 572, lr : 0.00064
Train Loss: 0.6652, Train Acc: 61.10%, Train profit : 7984.15%
Val Loss: 0.6707, Val Acc: 63.12%, Val profit : 7608.88%
Epoch: 30, Batch: 715, lr : 0.00064
Train Loss: 0.6652, Train Acc: 61.34%, Train profit : 7467.42%
Val Loss: 0.6675, Val Acc: 62.68%, Val profit : 7606.31%
Epoch: 30, Batch: 858, lr : 0.00064
Train Loss: 0.6648, Train Acc: 60.72%, Train profit : 6462.40%
Val Loss: 0.6683, Val Acc: 63.57%, Val profit : 7607.95%
Epoch: 30, Batch: 1001, lr : 0.00064
Train Loss: 0.6670, Train A

In [6]:
torch.save(model.state_dict(), './model/layer6_new/Transformer_layer6_4.pt')